# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
#%%bash 
#pip3 install duckdb
#pip3 install jupysql
#pip3 install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

There's a new jupysql version available (0.10.0), you're running 0.9.1. To upgrade: pip install jupysql --upgrade


Y ya podemos usar **SQL** dentro de una notebook!

Para cargar el archivo `.csv` a una tabla:

In [43]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('/home/maxibeckel/maestria_datos/dmeyf/dmeyf2023/data/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [31]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06,CONTINUA
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36,CONTINUA
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09,CONTINUA
3,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90,CONTINUA
4,29187499,202103,1,0,0,71,375,4189.56,54397.66,4212.77,...,7,0.00,-18616.76,0.00,8386,1891.57,1,0,727.26,CONTINUA


In [9]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [44]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS
with clase_ternaria as (
    select
        numero_de_cliente,
        MIN(foto_mes) AS lowest_foto_mes,
        MAX(foto_mes) AS highest_foto_mes,
        case 
            when highest_foto_mes = lowest_foto_mes AND lowest_foto_mes = '202103' then 'BAJA+1'
            when highest_foto_mes = lowest_foto_mes+1 AND lowest_foto_mes = '202103' then 'BAJA+2'
            when highest_foto_mes = lowest_foto_mes+2 AND lowest_foto_mes = '202103' then 'CONTINUA'
            -- else 'OTHER'
            end AS clase_ternaria
    from competencia_01
    group by 
    numero_de_cliente
) select
    cp.*,
    clase_ternaria
from competencia_01 cp
join clase_ternaria p using(numero_de_cliente)
        

,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [45]:
%%sql
SELECT
clase_ternaria, COUNT(clase_ternaria) AS n, ROUND( COUNT(clase_ternaria) * 100.0 / SUM(COUNT(clase_ternaria)) OVER (), 2) AS porcentaje
FROM
(
    SELECT 
    numero_de_cliente, clase_ternaria
    FROM competencia_01
    GROUP BY numero_de_cliente, clase_ternaria 
)
GROUP BY clase_ternaria
ORDER BY clase_ternaria

,clase_ternaria,n,porcentaje
0,BAJA+1,1049,0.64
1,BAJA+2,963,0.59
2,CONTINUA,161312,98.77
3,NaN,0,0.00


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [46]:
%%sql
COPY competencia_01 TO '/home/maxibeckel/maestria_datos/dmeyf/dmeyf2023/data/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo